In [92]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
import wikipedia
# import wikipediaapi 
import datetime
import pandas as pd
import numpy as np
from sklearn import feature_extraction
from sklearn import model_selection
import nltk
from nltk import tokenize
from nltk import word_tokenize, pos_tag
import string
from keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import multiclass
from sklearn import metrics
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [94]:
classes = ['sport', 'economy', 'politics', 'health', 'tourism', 
           'entertainment', 'culture', 'music', 'education', 
           'real estate', 'nutrition', 'science', 'history', 'technology'
          ]

In [95]:
def generate_data(classes, n):
    data = [];
    for i in classes:
#         topics = wikipedia.search(i, results = n)
#         for j in topics:
#         print(classes)
#         print("Class: ", i)
        for j in list(wikipedia.search(i,results = n)):
            wiki_wiki = wikipediaapi.Wikipedia('en')
            page_py = wiki_wiki.page(j)
            if(page_py.exists()):
#                 print("Class: ", i)
#                 print("Title: ", j)
#                     print("Section: ", page_py.sections[0].title)
#                     print("Text: ", page_py.text)
                for k in page_py.categories:
                    if i in k.lower(): 
#                         print("Class: ", i)
                        data.append((i, page_py.summary))
                        break
            else:
                continue
    return data

In [96]:
# data = generate_data(classes, 10)

In [97]:
# for i in classes:
#     c = 0
#     for j in range(0, len(data)):
#         if(data[j][0] == i):
#             c = c + 1
#     print("%s %d" %(i, c))

In [98]:
def create_data_csv_file(name, data):
    name = name + '.csv'
#         '_' + str(datetime.datetime.now()).replace(' ', '_').replace('.', '-').replace(':', '-') + 
#         '.csv'
    sep = '<>'
    cols = 'class' + sep + 'text\n'
    with open(name, 'w+', encoding='utf-8') as file:
        file.write(cols)
#         YELLOW,SMALL,STRETCH,ADULT,T
        for i in range(0, len(data)):
            line = data[i][0] + sep + data[i][1].replace('\n', ' ')

            file.write(line)
            file.write('\n')
    file.close()
            
#     print(text)

In [99]:
# create_data_csv_file('data_10', data)

In [100]:
df = pd.read_csv('data_10.csv', sep = '<>')
df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,class,text
0,sport,Sport pertains to any form of competitive phys...
1,sport,Sport was an American sports magazine. Launche...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп..."
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...
4,sport,Sport diving is an underwater sport that uses ...
...,...,...
100,technology,"Fintech a portmanteau of ""financial technology..."
101,technology,TCL Technology (originally an abbreviation for...
102,technology,Space technology is technology for use in oute...
103,technology,A technology company (or tech company) is an e...


In [101]:
df['class'].value_counts()

sport            10
tourism           9
culture           9
education         9
real estate       9
nutrition         8
science           8
technology        8
health            7
music             7
politics          6
history           6
entertainment     5
economy           4
Name: class, dtype: int64

In [102]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [103]:
dict_class_index = {
    class_name: i for i, class_name in enumerate(df['class'].unique())
}

df['class num'] = df['class'].map(dict_class_index)

In [104]:
df

,class,text,class num
0,sport,Sport pertains to any form of competitive phys...,0
1,sport,Sport was an American sports magazine. Launche...,0
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0
4,sport,Sport diving is an underwater sport that uses ...,0
...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13
101,technology,TCL Technology (originally an abbreviation for...,13
102,technology,Space technology is technology for use in oute...,13
103,technology,A technology company (or tech company) is an e...,13


In [105]:
dict_class_index

{'sport': 0,
 'economy': 1,
 'politics': 2,
 'health': 3,
 'tourism': 4,
 'entertainment': 5,
 'culture': 6,
 'music': 7,
 'education': 8,
 'real estate': 9,
 'nutrition': 10,
 'science': 11,
 'history': 12,
 'technology': 13}

In [106]:
!pip install advertools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import advertools as adv
stop_words = sorted(adv.stopwords['english'])
print(stop_words)

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon

In [108]:
def clear_text(text):
    new_text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])).str.lower()
    new_text = new_text.replace(r'[^\w\s]+', '', regex=True)
    return new_text

In [109]:
new_text = clear_text(df['text'])

In [110]:
df['clear text'] = new_text
df

,class,text,class num,clear text
0,sport,Sport pertains to any form of competitive phys...,0,sport pertains form competitive physical activ...
1,sport,Sport was an American sports magazine. Launche...,0,sport american sports magazine launched septem...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0,sport sport sport russian спорт спорт спорт 19...
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0,sport lisboa e benfica portuguese pronunciatio...
4,sport,Sport diving is an underwater sport that uses ...,0,sport diving underwater sport uses recreationa...
...,...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13,fintech portmanteau financial technology refer...
101,technology,TCL Technology (originally an abbreviation for...,13,tcl technology originally abbreviation telepho...
102,technology,Space technology is technology for use in oute...,13,space technology technology use outer space tr...
103,technology,A technology company (or tech company) is an e...,13,a technology company or tech company electroni...


In [111]:
newest_text = []
for i in range(0, len(df)):
    newest_text.append(word_tokenize(df['clear text'][i]))

In [112]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [113]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [114]:
def my_lemmitezer(text):
    return pos_tag(word_tokenize(text))

In [115]:
tokenized_lemmitzed_text = []
for i in range(0, len(df)):
    tokenized_lemmitzed_text.append(my_lemmitezer(df['clear text'][i]))

In [116]:
df['tokenized lemmitzed text'] = tokenized_lemmitzed_text

In [117]:
stemmer = nltk.stem.PorterStemmer()

In [118]:
def my_stemmer(text):
    tokenized_stemmed_text = []
    tokens = word_tokenize(text)
    for token in tokens:
        tokenized_stemmed_text.append(stemmer.stem(token))
    return tokenized_stemmed_text

In [119]:
tokenized_stemmed_text = []
for i in range(0, len(df)):
    tokenized_stemmed_text.append(my_stemmer(df['clear text'][i]))

In [120]:
df['tokenized stemmed text'] = tokenized_stemmed_text
# df['tokenized stemmed text'] = df['tokenized stemmed text'].str.join(' ')

In [121]:
x = df['clear text']
x

0      sport pertains form competitive physical activ...
1      sport american sports magazine launched septem...
2      sport sport sport russian спорт спорт спорт 19...
3      sport lisboa e benfica portuguese pronunciatio...
4      sport diving underwater sport uses recreationa...
                             ...                        
100    fintech portmanteau financial technology refer...
101    tcl technology originally abbreviation telepho...
102    space technology technology use outer space tr...
103    a technology company or tech company electroni...
104    agricultural technology agrotechnology abbrevi...
Name: clear text, Length: 105, dtype: object

In [122]:
y = df['class num']
y

0       0
1       0
2       0
3       0
4       0
       ..
100    13
101    13
102    13
103    13
104    13
Name: class num, Length: 105, dtype: int64

In [123]:
x_tr, x_ft, y_tr, y_ft = model_selection.train_test_split(x, y, test_size=0.2, stratify=y)

In [124]:
x_tr

22     mental health encompasses emotional psychologi...
21     health care healthcare improvement health prev...
42     popular culture also called mass culture pop c...
38     sony interactive entertainment sie known sony ...
28     sex tourism refers practice traveling foreign ...
                             ...                        
65     the montessori method education system educati...
100    fintech portmanteau financial technology refer...
63     physical education abbreviated phys ed pe subj...
16     a political party organization coordinates can...
96     the history christianity concerns christian re...
Name: clear text, Length: 84, dtype: object

In [125]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_tr, y_tr, test_size=0.3, stratify=y_tr, random_state=5)

In [126]:
count_vectorizer_t = feature_extraction.text.TfidfVectorizer()

In [127]:
count_vectorizer_t.fit(x_train)

TfidfVectorizer()

In [128]:
x_train_t_tfidf = count_vectorizer_t.transform(x_train)
x_test_t_tfidf = count_vectorizer_t.transform(x_test)

In [129]:
model = svm.LinearSVC(max_iter=100000)

In [130]:
model_t = multiclass.OneVsRestClassifier(model)

In [131]:
model_t.fit(x_train_t_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [132]:
y_test_predicted_t = model_t.predict(x_test_t_tfidf)

In [133]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_t), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,2,0,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [134]:
print(metrics.classification_report(y_test, y_test_predicted_t))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.67      1.00      0.80         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       0.00      0.00      0.00         2
          13       1.00      1.00      1.00         2

    accuracy                           0.92        26
   macro avg       0.88      0.93      0.90        26
weighted avg       0.87      0.92      0.89        26



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [135]:
metrics.f1_score(y_test, y_test_predicted_t, average='macro')

0.9000000000000001

In [136]:
count_vectorizer_l = feature_extraction.text.TfidfVectorizer(tokenizer=my_lemmitezer)

In [137]:
count_vectorizer_l.fit(x_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(tokenizer=<function my_lemmitezer at 0x7f0031ce7a70>)

In [138]:
x_train_l_tfidf = count_vectorizer_l.transform(x_train)
x_test_l_tfidf = count_vectorizer_l.transform(x_test)

In [139]:
model = svm.LinearSVC(max_iter=100000)

In [140]:
model_l = multiclass.OneVsRestClassifier(model)

In [141]:
model_l.fit(x_train_l_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [142]:
y_test_predicted_l = model_l.predict(x_test_l_tfidf)

In [143]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_l), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,2,0,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [144]:
print(metrics.classification_report(y_test, y_test_predicted_l))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.67      1.00      0.80         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       0.00      0.00      0.00         2
          13       1.00      1.00      1.00         2

    accuracy                           0.92        26
   macro avg       0.88      0.93      0.90        26
weighted avg       0.87      0.92      0.89        26



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [145]:
metrics.f1_score(y_test, y_test_predicted_l, average='macro')

0.9000000000000001

In [146]:
count_vectorizer_s = feature_extraction.text.TfidfVectorizer(tokenizer=my_stemmer)

In [147]:
count_vectorizer_s.fit(x_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(tokenizer=<function my_stemmer at 0x7f0031cebe60>)

In [148]:
x_train_s_tfidf = count_vectorizer_s.transform(x_train)
x_test_s_tfidf = count_vectorizer_s.transform(x_test)

In [149]:
model = svm.LinearSVC(max_iter=100000)

In [150]:
model_s = multiclass.OneVsRestClassifier(model)

In [151]:
model_s.fit(x_train_s_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [152]:
y_test_predicted_s = model_s.predict(x_test_s_tfidf)

In [153]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_s), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,2,0,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [154]:
print(metrics.classification_report(y_test, y_test_predicted_s))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       0.67      1.00      0.80         2
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       0.00      0.00      0.00         2
          13       1.00      1.00      1.00         2

    accuracy                           0.92        26
   macro avg       0.88      0.93      0.90        26
weighted avg       0.87      0.92      0.89        26



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [155]:
metrics.f1_score(y_test, y_test_predicted_s, average='macro')

0.9000000000000001

In [156]:
# !wget https://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

In [157]:
file = open('glove.6B.100d.txt', 'r', encoding='utf-8')
lines  = file.readlines()
file.close()

In [158]:
word_embeddings = {}

for line in lines:
    strs = line.split()
    word = strs[0]
    coefs = np.array(strs[1:], dtype='float32')
    
    word_embeddings[word] = coefs

In [159]:
x_train

40     caesars entertainment inc eldorado resorts inc...
58     postgraduate education graduate education nort...
68     real estate development property development b...
81     clinical nutrition centers prevention diagnosi...
31     the minister tourism head governmental departm...
75     nutrition biochemical physiological process or...
64     sex education known sexual education sexuality...
78     saprotrophic nutrition lysotrophic nutrition p...
72     real estate appraisal property valuation land ...
16     a political party organization coordinates can...
1      sport american sports magazine launched septem...
52     country also called country western genre popu...
51     music music music put another nickel in popula...
35     urban tourism called city tourism form tourism...
5      athletics group sporting events involves compe...
7      squash racket ball sport played players fourwa...
12     the peoples republic china high income develop...
29     space tourism human spac

In [160]:
dimension = 100
# voc_size = 15000 #voditi racuna ovo mora da se povecava kada se poveca broj tekstova
voc_size = len(word_embeddings)
tokenizer = Tokenizer(num_words=voc_size)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [161]:
num_of_words = min(voc_size, len(word_index))

In [162]:
coef_matrix = np.zeros((num_of_words, dimension))

for word, ord_num in word_index.items():
    if ord_num >= num_of_words:
        continue
    
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        coef_matrix[ord_num] = embedding_vector

In [163]:
# max_len = max([len(i) for i in x_train])
max_len = 100

In [164]:
data_len = len(x_train)
x_train = x_train.apply(lambda x: [coef_matrix[word_index[word] - 1] for word in x.split(' ') if word in word_index])
x_train_glove = sequence.pad_sequences([x for x in x_train], maxlen=max_len, dtype='float16')
x_train_glove = x_train_glove.reshape(data_len, max_len * 100).astype('float16')

In [165]:
x_train_glove

array([[ 0.     ,  0.     ,  0.     , ..., -0.03128, -0.0627 ,  0.1418 ],
       [ 0.     ,  0.     ,  0.     , ..., -0.966  ,  0.7915 ,  0.12445],
       [-0.1913 ,  0.2834 ,  0.6074 , ...,  0.4731 ,  0.11115, -1.206  ],
       ...,
       [-0.2974 ,  0.663  ,  0.0799 , ..., -0.1459 ,  0.8276 ,  0.2705 ],
       [-0.7764 , -0.647  ,  0.09607, ..., -0.4229 ,  0.586  , -0.424  ],
       [ 0.09863,  0.01614, -0.6143 , ..., -0.1508 , -0.4663 , -0.662  ]],
      dtype=float16)

In [166]:
data_len = len(x_test)
x_test = x_test.apply(lambda x: [coef_matrix[word_index[word] - 1] for word in x.split(' ') if word in word_index])
x_test_glove = sequence.pad_sequences([x for x in x_test], maxlen=max_len, dtype='float16')
x_test_glove = x_test_glove.reshape(data_len, max_len * 100).astype('float16')

In [167]:
# def convert_to_svm_fromat(text, max_len):
#     text = text.apply(lambda x: [coef_matrix[word_index[word] - 1] for word in x.split(' ') if word in word_index])

#     x_train_glove = []

#     for i in text:
#         glove = []
#         for j in i:
#             glove.extend(j)
#         if len(i) < max_len:
#             glove.extend(np.zeros((max_len - len(i))*100))
#         x_train_glove.append(glove)
#     return x_train_glove

In [168]:
model = svm.LinearSVC(max_iter=100000)

In [169]:
model_g = multiclass.OneVsRestClassifier(model)

In [170]:
model_g.fit(x_train_glove, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [171]:
y_test_predicted_g = model_g.predict(x_test_glove)

In [172]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_g), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,1,0,0,0,0,0,1,0,0,0,0,0,0,0
economy,1,0,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,0,0,0,0,0,0,2,0,0,0,0,0
health,0,0,0,0,0,0,0,0,0,0,2,0,0,0
tourism,0,0,0,0,0,0,0,0,1,0,0,0,0,1
entertainment,1,0,0,0,0,0,0,0,0,0,0,0,0,0
culture,0,0,0,0,1,0,0,0,0,1,0,0,0,0
music,0,0,0,0,1,0,0,0,1,0,0,0,0,0
education,0,0,0,0,0,0,0,0,1,0,0,0,0,1
real estate,0,0,0,0,0,0,0,0,1,1,0,0,0,0


In [174]:
print(metrics.classification_report(y_test, y_test_predicted_g))

              precision    recall  f1-score   support

           0       0.25      0.50      0.33         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         2
           8       0.12      0.50      0.20         2
           9       0.50      0.50      0.50         2
          10       0.33      0.50      0.40         2
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         2

    accuracy                           0.15        26
   macro avg       0.09      0.14      0.10        26
weighted avg       0.09      0.15      0.11        26



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [175]:
metrics.f1_score(y_test, y_test_predicted_g, average='macro')

0.10238095238095236